In [5]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [275]:
# make file into pandas dataframe and get median

def process_income(month):
    income_data = pd.read_csv(f'data_income_unemployment/household_income_{month}.csv')#,
                                # for monthly data only
                                # header = 4)

    # find median "person number" per state
    total_people_per_state = income_data.sum(axis = 1)
    middle_person = total_people_per_state/2
    # middle_person = pd.DataFrame(middle_person)
    # print(middle_person)

    # delete text column; not used for newer weekly data, though it doesn't really matter.
    # states_removed = income_data.drop(['Selected Geographies'], axis = 1).astype(float)
    # print(states_removed)

    # get cumulative sum of people across rows
    # for monthly data set, use this:
    # states_cumulative = np.cumsum(states_removed, axis = 1)

    # for weekly data set, use this:
    states_cumulative = np.cumsum(income_data, axis = 1)

    # print(states_cumulative)

    # print("cumulative states type is ",type(states_cumulative))
    # print("middle person type is ",type(middle_person))
    
    # find where the median lies within income brackets
    
    def find_median(month):
        i = 0
        medians_across_states = pd.DataFrame(columns = [f'median_income_{month}'])

        # for monthly set, use this:
        for i in range(51):
            j = 0
            # print("j has been reset to", j)

            # while half the total sample for the state is >= cumulative numbers, check the next cumulative number in the income bracket.
            while (middle_person.iloc[i] > states_cumulative.iloc[i,j]) or (middle_person.iloc[i] == states_cumulative.iloc[i,j]):
                median_column = states_cumulative.columns[j]
                
                # statements to check code
                '''
                print("boolean is ", (middle_person.iloc[i] > states_cumulative.iloc[i,j]) or (middle_person.iloc[i] == states_cumulative.iloc[i,j]))
                print("middle_person.iloc[i] = ", middle_person.iloc[i])
                print("states_cumulative.iloc[i,j] = ", states_cumulative.iloc[i,j])
                '''
                
                j += 1

            else:
                median_column = states_cumulative.columns[j]
                # print("The median column is {} for i = {} and j = {}".format(median_column, i, j))
                # print("Median column is ", type(median_column))

                # add into dataframe
                medians_across_states = medians_across_states.append({f'median_income_{month}': median_column},
                                                                        ignore_index = True)

                # print(medians_across_states)

        return medians_across_states

    # print(income_data)

    # put into new dataframe with just states, median income, and month

    return find_median(month)

In [276]:
# process_income('apr').to_csv('apr.csv', index = False)

process_income('w1')

,median_income_w1
0,"$50,000 - $74,999"
1,"$50,000 - $74,999"
2,"$50,000 - $74,999"
3,"$35,000 - $49,999"
4,"$50,000 - $74,999"
5,"$75,000 - $99,999"
6,"$50,000 - $74,999"
7,"$50,000 - $74,999"
8,"$75,000 - $99,999"
9,"$50,000 - $74,999"


In [238]:
states_acronyms = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
    
state_codes = ["01","02","04","05","06","08","09","10","11","12","13","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","44","45","46","47","48","49","50","51","53","54","55","56"]

def states_list():
    states_list = pd.read_csv(f'data_income_unemployment/states_list.csv', header = 0)
    states_list["abbreviations"] = states_acronyms
    states_list["FIPS_code"] = state_codes

    print(states_list)
    return states_list

# use this to generate months data
joined_data = states_list().join(process_income('mar'), lsuffix='_caller', rsuffix='_other')
joined_data.to_csv('mar.csv', index=False)

 abbreviations FIPS_code
0                Alabama            AL        01
1                 Alaska            AK        02
2                Arizona            AZ        04
3               Arkansas            AR        05
4             California            CA        06
5               Colorado            CO        08
6            Connecticut            CT        09
7               Delaware            DE        10
8   District of Columbia            DC        11
9                Florida            FL        12
10               Georgia            GA        13
11                Hawaii            HI        15
12                 Idaho            ID        16
13              Illinois            IL        17
14               Indiana            IN        18
15                  Iowa            IA        19
16                Kansas            KS        20
17              Kentucky            KY        21
18             Louisiana            LA        22
19                 Maine            ME      

In [196]:
# opening new types of data.
def open_excel_file(week, state):
    new_stats = pd.read_excel(f"data_income_unemployment/US_employment_stats_{week}.xlsx", sheet_name = state, usecols = "A:B", skiprows = 42)

    # cleaning and transposing data
    new_stats = new_stats.drop([8]).T
    new_stats = new_stats.reset_index()

    # fixing columns
    new_stats.columns = new_stats.iloc[0]
    new_stats = new_stats.drop(new_stats.index[0])
    new_stats = new_stats.drop('Household income ', axis = 1).reset_index(drop = True)

    return new_stats


In [197]:
open_excel_file('w1','AL')

,"Less than $25,000","$25,000 - $34,999","$35,000 - $49,999","$50,000 - $74,999","$75,000 - $99,999","$100,000 - $149,999","$150,000 - $199,999","$200,000 and above"
0,587836,441336,383656,701761,326753,399938,176707,73169


In [247]:
# putting data together to make it look like the other sets of data

# making it blank first to not be affected by past operations
data_per_state = pd.DataFrame()

for i in range(51):
    data_per_state = data_per_state.append(open_excel_file('w3',states_acronyms[i]), ignore_index = True)

data_per_state.to_csv('household_income_w3.csv', index=False)

# note that week 1 = Apr 23 - May 5
# week 2 = May 7 - May 12
# week 3 = May 14 - May 19

In [279]:
# again for new weekly data

joined_data = states_list().join(process_income('w3'), lsuffix='_caller', rsuffix='_other')
joined_data.to_csv('w3.csv', index=False)

Selected Geographies abbreviations FIPS_code
0                Alabama            AL        01
1                 Alaska            AK        02
2                Arizona            AZ        04
3               Arkansas            AR        05
4             California            CA        06
5               Colorado            CO        08
6            Connecticut            CT        09
7               Delaware            DE        10
8   District of Columbia            DC        11
9                Florida            FL        12
10               Georgia            GA        13
11                Hawaii            HI        15
12                 Idaho            ID        16
13              Illinois            IL        17
14               Indiana            IN        18
15                  Iowa            IA        19
16                Kansas            KS        20
17              Kentucky            KY        21
18             Louisiana            LA        22
19                 Maine